In [1]:
import natasha
import pymorphy2  # библиотека для морфологического анализа русского языка
                  # является одной из зависимостей natasha
import pandas as pd
from natasha import NamesExtractor, MorphVocab
import torch.nn as nn
import torch 
from slovnet.model.emb import NavecEmbedding
from sklearn.metrics import accuracy_score, roc_auc_score
import torch.nn.functional as F

In [2]:
df = pd.read_csv(r"C:\Users\User\Downloads\lenta\lenta\lenta-subset.csv")
df.head(5)

,url,title,text,topic,tags,date
0,https://lenta.ru/news/1999/10/04/tv/,Телеканалы станут вещать по единому тарифу,С 1 января 2000 года все телеканалы будут опла...,Экономика,Все,1999/10/04
1,https://lenta.ru/news/1999/10/04/volkswagen/,"Volkswagen выкупает остатки акций ""Шкоды""",Германский автопромышленный концерн Volkswagen...,Экономика,Все,1999/10/04
2,https://lenta.ru/news/1999/10/04/tumen/,Прибыль Тюменнефтегаза возросла в 10 раз,"Нераспределенная прибыль ОАО ""Тюменнефтегаз"", ...",Экономика,Все,1999/10/04
3,https://lenta.ru/news/1999/10/05/sprint/,Крупнейшее в истории слияние компаний происход...,Две крупнейших телекоммуникационных компании С...,Экономика,Все,1999/10/05
4,https://lenta.ru/news/1999/10/05/volga/,ГАЗ получил четверть обещанного кредита,"ОАО ""ГАЗ"" и Нижегородский банк Сбербанка Росси...",Экономика,Все,1999/10/05


In [3]:
def text_to_ids(emb, text: str, length: int) -> torch.tensor:
    """Преобразование строки в тензор с номерами токенов."""
    # Пунктуационные токены (их кодировать не будем)
    punct = [',', '.', ';', ':', '-', '...', '!', '?']
    # Проведем токенизацию текста
    d = natasha.Doc(text)
    d.segment(segmenter)
    # Для каждого токена, который найдется в словаре эмбеддингов подставим
    # его номер, для прочих подставим номер <unk>
    tmp = torch.tensor([emb.vocab.get(x.text.lower(), emb.vocab.unk_id)
                       for x in d.tokens
                       if x.text not in punct][:length])
    # Дополним последовательность (спереди) токенами <pad>
    return torch.nn.functional.pad(tmp, (length - len(tmp), 0), "constant", emb.vocab.pad_id)

In [4]:
emb = natasha.NewsEmbedding()
syntax_parser = natasha.NewsSyntaxParser(emb)
segmenter = natasha.Segmenter()

In [5]:
class_mapping = {"Культура": 0, "Спорт": 1, "Экономика": 2}
df["target"] = df["topic"].map(class_mapping)

In [6]:
df.dropna(inplace=True)

In [7]:
df = df.sample(frac=1.).reset_index(drop=True)
df_test = df[:1000].copy()
df_train = df[1000:].copy().reset_index(drop=True)

In [8]:
X = torch.stack([text_to_ids(emb, x, 50) for x in df.text], 0)
y = torch.tensor(df.target, dtype=torch.long)

In [9]:
X_test = torch.stack([text_to_ids(emb, x, 50) for x in df_test.text], 0)
y_test = torch.unsqueeze(torch.tensor(df_test.target, dtype=torch.long), 1)

In [10]:
data_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X, y), batch_size=8)
test_data_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(X_test, y_test), batch_size=16, shuffle=False)

In [11]:
class ImprovedLSTMClassifier(nn.Module):
    """Улучшенный классификатор текста с многоклассовой классификацией."""

    def __init__(self, num_classes):
        super().__init__()
        self.embedding = NavecEmbedding(emb)
        self.lstm = nn.LSTM(300, 30, batch_first=True)
        self.fc = nn.Linear(30, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        x = self.fc(x)
        return x

In [12]:
num_classes = 3  # Ваше количество классов
model = ImprovedLSTMClassifier(num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), 1e-3)

c:\Python310\lib\site-packages\slovnet\model\emb.py:46: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ..\torch\csrc\utils\tensor_numpy.cpp:212.)
  torch.from_numpy(navec.pq.indexes),


In [13]:
for epoch in range(5):
    model.train()
    for X, y in data_loader:
        optimizer.zero_grad()
        pred_logits = model(X)
        loss = criterion(pred_logits, y.squeeze())
        loss.backward()
        optimizer.step()
    
    model.eval()
    preds = []
    with torch.no_grad():
        for X, y in test_data_loader:
            pred_logits = model(X)
            preds.append(pred_logits)
    preds = torch.cat(preds)
    
    # Определение предсказанных классов
    preds_classes = preds.argmax(dim=1)
    
    roc_auc = roc_auc_score(df_test.target, F.softmax(preds, dim=1).numpy(), multi_class='ovr')
    print(f'Epoch {epoch}: train loss={loss.detach().item():.4f}. ' \
        f'Test accuracy {accuracy_score(df_test.target, preds_classes):.4f} ' \
        f'ROC_AUC {roc_auc:.4f}')

Epoch 0: train loss=0.0015. Test accuracy 0.9840 ROC_AUC 0.9992
Epoch 1: train loss=0.0005. Test accuracy 0.9880 ROC_AUC 0.9997
Epoch 2: train loss=0.0001. Test accuracy 0.9920 ROC_AUC 0.9999
Epoch 3: train loss=0.0000. Test accuracy 0.9940 ROC_AUC 0.9999
Epoch 4: train loss=0.0001. Test accuracy 0.9940 ROC_AUC 0.9999


Как мы видимо на тестовой обучающей выборке результат модели очень хороший. Теперь попробуй проверить модель на тестовой выборке.  

In [14]:
model.eval()
all_preds = []

with torch.no_grad():
    for X, y in test_data_loader:
        pred_logits = model(X)
        preds = F.softmax(pred_logits, dim=1)
        all_preds.append(preds)

all_preds = torch.cat(all_preds)

# Определение предсказанных классов
df_test['predicted'] = all_preds.argmax(dim=1).numpy()

In [15]:
df_test.head(5)

,url,title,text,topic,tags,date,target,predicted
0,https://lenta.ru/news/2009/02/05/biathlon1/,Попавшихся на допинге биатлонистов отстранили ...,Международный союз биатлонистов (IBU) временно...,Спорт,Все,2009/02/05,1,1
1,https://lenta.ru/news/2010/11/02/diagnose/,Рамзан Кадыров назвал писателя Германа Садулае...,Глава Чеченской республики Рамзан Кадыров в бе...,Культура,Все,2010/11/02,0,0
2,https://lenta.ru/news/2010/05/20/referees/,Названы судьи матча Россия — Канада,Международная федерация хоккея (IIHF) обнародо...,Спорт,Все,2010/05/20,1,1
3,https://lenta.ru/news/2011/07/01/lineup/,"Футболист ""Спартака"" попал в стартовый состав ...","Футболист московского ""Спартака"" Маркос Рохо п...",Спорт,Все,2011/07/01,1,1
4,https://lenta.ru/news/2017/03/27/deadboxing/,Шотландский боксер умер из-за сгонки веса,"Шотландец Джордан Коу умер, пытаясь сбросить в...",Спорт,Бокс и ММА,2017/03/27,1,1


In [16]:
errors_df= df_test[df_test['target'] != df_test['predicted']]

In [17]:
errors_df

,url,title,text,topic,tags,date,target,predicted
346,https://lenta.ru/news/2017/06/16/filmzenittv/,По ТВ покажут фильм о матче «Зенита» в Северно...,На телеканале НТВ состоится премьера документа...,Спорт,Футбол,2017/06/16,1,0
459,https://lenta.ru/news/2016/07/19/shcwartz/,Шварценеггер предложил покурить сигару полутор...,Голливудский актер и культурист Арнольд Шварце...,Спорт,Бокс и ММА,2016/07/19,1,0
522,https://lenta.ru/news/2017/04/17/shnurska/,Шнуров спел про слово из трех букв в честь поб...,"Российский музыкант, лидер группы «Ленинград» ...",Спорт,Хоккей,2017/04/17,1,0
553,https://lenta.ru/news/2002/10/11/control/,"Степашин проверит, как Лужков использует памят...",Депутаты Госдумы обратились в Счетную палату Р...,Культура,Все,2002/10/11,0,2
743,https://lenta.ru/news/2006/05/24/library/,В Москве построили библиотеку украинской литер...,В центре Москвы открывается библиотека украинс...,Экономика,Все,2006/05/24,2,0
872,https://lenta.ru/news/2006/05/10/opera/,Петербургская консерватория закрыла театр,Помещения Театра оперы и балета Санкт-Петербур...,Экономика,Все,2006/05/10,2,0


Как мы видим результат очень хороший и только 6 строк было ошибочно определено